# Lab 02 — Terminología y Semántica

## Mini “Terminology Server” dentro de PostgreSQL

**Meta del lab:** comparar analítica **antes vs después** de codificar diagnósticos y laboratorios.


## 0) Conexión a la base de datos

Este notebook asume el mismo setup de Docker del curso (db + jupyter).

- Si no levantaste Docker:
  - Abre una terminal en `labs/lab-2-terminology-semantics/`
  - `docker compose up -d`


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

### Re-crear el modelo de datos de los laboratorios pasados

Vas a ejecutar el scripts SQL `sql/002_ehr_schema.sql` → crea tablas y llenar de datos.

> [!TIP]
> Ejecuta los scripts desde tu terminal (PowerShell) así:
>
> ```powershell
> Get-Content .\sql\002_ehr_schema.sql | docker compose exec -T db psql -U uvg_user -d health_data
> ```


### Ver qué tablas existen


In [ ]:
pd.read_sql("""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema='public'
ORDER BY table_name;
""", engine)

## 1) BEFORE: Analítica con texto libre (sin terminología)

Primero veremos el problema: _mismo concepto, textos distintos_.

### 1.1 Conteo de diagnósticos por texto (raw)


In [ ]:
pd.read_sql("""
SELECT diagnosis_text, COUNT(*) AS n
FROM diagnoses
GROUP BY diagnosis_text
ORDER BY n DESC, diagnosis_text;
""", engine)

**Observa:** si hay typos o variaciones, aparecen como categorías separadas.

### 1.2 Conteo de labs por label (raw)


In [ ]:
pd.read_sql("""
SELECT label, COUNT(*) AS n_items
FROM d_labitems
GROUP BY label
ORDER BY n_items DESC, label;
""", engine)

## 2) Crear el “mini terminology server” (tablas)

Vas a ejecutar 3 scripts SQL en orden:

1. `sql/003_terminology_schema.sql` → crea tablas
2. `sql/004_seed_terminology.sql` → llena códigos + sinónimos + mappings
3. `sql/005_apply_mappings.sql` → aplica los mappings al EHR

> [!TIP]
> Ejecuta los scripts desde tu terminal (PowerShell) así:
>
> ```powershell
> Get-Content .\sql\003_terminology_schema.sql | docker compose exec -T db psql -U uvg_user -d health_data
> ```
>
> Repite igual para 004 y 005.


### ✅ Checkpoint A

Cuando termines de ejecutar los scripts, corre la celda de abajo y confirma que ves estas tablas:

- `terminology_systems`
- `terminology_concepts`
- `map_diagnosis_text_to_concept`
- `map_labitem_to_concept`

> Si no aparecen, revisa que corriste `003_terminology_schema.sql` en la DB correcta.


In [ ]:
pd.read_sql("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema='public'
  AND table_name IN (
    'terminology_systems',
    'terminology_concepts',
    'map_diagnosis_text_to_concept',
    'map_labitem_to_concept'
  )
ORDER BY table_name;
""", engine)


## 3) Simular “API”: buscar conceptos y resolver mapeos

Un terminology service real expone APIs para:

- buscar conceptos por texto
- devolver el concepto “oficial” (system + code + display)
- resolver equivalencias / mapeos desde términos locales

Aquí lo simulamos con SQL.

### 3.1 Buscar en el catálogo de conceptos (por display)


In [ ]:
term = "wbc"  # prueba: 'sepsis', 'htn', 'pneumonia', 'creatinine', 'lactate'

pd.read_sql(
    """
    SELECT
      s.system_name,
      c.code,
      c.display,
      c.concept_type
    FROM terminology_concepts c
    JOIN terminology_systems s ON s.system_id = c.system_id
    WHERE LOWER(c.display) LIKE LOWER(%(q)s)
    ORDER BY s.system_name, c.code;
    """,
    engine,
    params={"q": f"%{term}%"},
)


## 4) AFTER: Analítica con códigos (semántica)

### 4.1 Diagnósticos por ICD-10 (ya codificados)


In [ ]:
pd.read_sql("""
SELECT code, code_display, COUNT(*) AS n
FROM diagnoses
WHERE code IS NOT NULL
GROUP BY code, code_display
ORDER BY n DESC, code;
""", engine)


### 4.2 ¿Qué diagnósticos quedaron SIN código?

Esto pasa en hospitales reales. Es parte del reto.


In [ ]:
pd.read_sql("""
SELECT diagnosis_text, COUNT(*) AS n
FROM diagnoses
WHERE code IS NULL
GROUP BY diagnosis_text
ORDER BY n DESC, diagnosis_text;
""", engine)


### 4.3 Labs por LOINC (ya codificados)


In [ ]:
pd.read_sql("""
SELECT code, code_display, COUNT(*) AS n_items
FROM d_labitems
WHERE code IS NOT NULL
GROUP BY code, code_display
ORDER BY n_items DESC, code;
""", engine)


## 5) Ejercicio: mejora la cobertura de codificación

### Tarea 5.1 — Completa mappings faltantes (diagnósticos)

1. Identifica diagnósticos sin código (celda 4.2)
2. Encuentra el concepto correcto en `terminology_concepts` (ICD-10 mini)
3. Inserta el mapping en `map_diagnosis_text_to_concept`
4. Re-ejecuta `sql/005_apply_mappings.sql`

> [!IMPORTANT]
> No inventes códigos: usa únicamente los del mini catálogo.


In [42]:
pd.read_sql("""
SELECT diagnosis_text, code
FROM diagnoses
ORDER BY diagnosis_id;
""", engine)

,diagnosis_text,code
0,Hypertension,I10
1,Chest pain,R07.9
2,Elective procedure follow-up,Z09
3,Sepsis,A41.9
4,Acute Kidney Injury,N17.9
5,Asthma exacerbation,J45.901
6,Pneumonia,J18.9
7,Trauma,T14.90
8,Urinary tract infection,N39.0
9,Dehydration,E86.0


In [43]:
pd.read_sql("""
SELECT COUNT(*) AS uncoded
FROM diagnoses
WHERE code IS NULL;
""", engine)

,uncoded
0,0


### Tarea 5.2 — Completa mappings faltantes (labs)

Haz lo mismo con `d_labitems` → LOINC mini:

1. Crea/usa un labitem local (label/unit)
2. Encuentra el concepto LOINC en `terminology_concepts`
3. Inserta el mapping en `map_labitem_to_concept`
4. Re-ejecuta `sql/005_apply_mappings.sql`


In [44]:
pd.read_sql("""
SELECT li.label, li.unit
FROM d_labitems li
LEFT JOIN map_labitem_to_concept m
  ON li.label = m.label
 AND (li.unit = m.unit OR (li.unit IS NULL AND m.unit IS NULL))
WHERE m.concept_id IS NULL;
""", engine)

,label,unit


In [46]:
pd.read_sql("""
SELECT label, unit, code, code_display
FROM d_labitems
ORDER BY labitem_id;
""", engine)

,label,unit,code,code_display
0,Creatinine,mg/dL,2160-0,Creatinine [Mass/volume] in Serum or Plasma
1,Hemoglobin,g/dL,718-7,Hemoglobin [Mass/volume] in Blood
2,White Blood Cells,10^9/L,6690-2,Leukocytes [#/volume] in Blood
3,Platelets,10^9/L,777-3,Platelets [#/volume] in Blood
4,Lactate,mmol/L,2524-7,Lactate [Moles/volume] in Blood


In [47]:
pd.read_sql("""
SELECT COUNT(*) AS uncoded
FROM d_labitems
WHERE code IS NULL;
""", engine)

,uncoded
0,0


## 6) Mini-analítica clínica: “Sepsis proxy” (con y sin terminología)

Definición didáctica (NO clínica real):

- WBC > 12 **o** Lactate > 2

### 6.1 Sin terminología: filtrar por labels


In [ ]:
pd.read_sql("""
WITH piv AS (
  SELECT
    le.hadm_id,
    MAX(CASE WHEN li.label = 'White Blood Cells' THEN le.value_num END) AS max_wbc,
    MAX(CASE WHEN li.label = 'Lactate' THEN le.value_num END) AS max_lactate
  FROM labevents le
  JOIN d_labitems li ON li.labitem_id = le.labitem_id
  GROUP BY le.hadm_id
)
SELECT *
FROM piv
WHERE max_wbc > 12 OR max_lactate > 2
ORDER BY hadm_id;
""", engine)

### 6.2 Con terminología: filtrar por LOINC (más robusto)


In [ ]:
pd.read_sql("""
WITH piv AS (
  SELECT
    le.hadm_id,
    MAX(CASE WHEN li.code = '6690-2' THEN le.value_num END) AS max_wbc,
    MAX(CASE WHEN li.code = '2524-7' THEN le.value_num END) AS max_lactate
  FROM labevents le
  JOIN d_labitems li ON li.labitem_id = le.labitem_id
  GROUP BY le.hadm_id
)
SELECT *
FROM piv
WHERE max_wbc > 12 OR max_lactate > 2
ORDER BY hadm_id;
""", engine)


## 7) Reflexión (para escribir en el PR)

Responde en el texto del PR:

1. ¿Qué cambió en la analítica al pasar de texto libre → códigos? (1–3 frases)
2. ¿Por qué no basta con “guardar el nombre” del lab o diagnóstico?
3. ¿Qué te faltaría para que esto sea un “terminology server” real (en producción)?

> [!TIP]
> Piensa en: API, versiones, mappings oficiales, validación, gobernanza, logs.
